버전 확인

In [1]:
import tensorflow 

print(tensorflow.__version__)

2.8.2


# 데이터 읽어오기

In [2]:
import glob
import os , re
import tensorflow as tf
import numpy as np
txt_file_path = '/content/drive/MyDrive/aiffel csv/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus =[]

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me']


# 데이터 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1. 양쪽 공백제거 및 모두 소문자로 변환

    sentence = re.sub(r"([?.!,¿])", r"  \1 ", sentence)  #2 특수문자 양쪽 공백 넣기

    sentence = re.sub(r'[" "]+', " ", sentence)  #3 여러개의 공백을 하나의 공백

    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4 a-zA_Z?.!,¿ 가 아닌 모든 문자를 하나의 공백을 바꿈
    
    sentence = sentence.strip() #5 다시 양쪽 공백을 지움

    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start> , 끝에는 <end> 추가

    return sentence
# 문장 확인
print(preprocess_sentence(" This @_is ;;;sample      Sentence. "))
    

<start> this is sample sentence . <end>


In [26]:
corpus = []

for sentence in raw_corpus:
    
    if len(sentence) == 0: continue # 우리가 원하지 않는 문장 건너뛰기
    
    preprocessed_sentence = preprocess_sentence(sentence) 
    
    if len(preprocessed_sentence.split()) > 15: continue  # 토큰의 개수 15개 넘어가는 문장 데이터 제외하기
    corpus.append(preprocessed_sentence)

print(len(corpus))
corpus[:10]

156227


['<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 '<start> i ll never be untrue oh baby <end>',
 '<start> let s , let s stay together gether <end>',
 '<start> lovin you whether , whether <end>',
 '<start> times are good or bad , happy or sad <end>']

In [5]:
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다

    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 570   7 ...   0   0   0]
 [  2  26  25 ...   0   0   0]
 [  2  66   7 ...   0   0   0]
 ...
 [  2  30  19 ...   1   3   0]
 [  2 344  19 ...   3   0   0]
 [  2  41 129 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fd5657d3b10>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


# 훈련 데이터셋과 평가 데이터셋 분리

In [7]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]  #처음~마지막 -1
tgt_input = tensor[:, 1:]  #처음+1~마지막

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=30)


In [8]:
print("Source.Train:", enc_train.shape)

Source.Train: (124981, 14)


In [15]:
BUFFER_SIZE = len(src_input) # source senten 수
BATCH_SIZE = 128 # 한 번 학습할 데이터 수


steps_per_eopch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 #왜 1을 더해줬을까나 ,tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
# VOCAB_SIZE : 벡터화한 단어 개수

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(128, 14), dtype=tf.int32, name=None), TensorSpec(shape=(128, 14), dtype=tf.int32, name=None))>

# 모델 구성

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        #embedding_size는 word vector의 차원수이다.
        #hidden size는 LSTM레이어의 hidden state의 차원수이다 
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 128 
hidden_size =256
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [17]:
for src_sample, tgt_sample in dataset.take(1): break
# 데이터셋에서 데이터 한 배치만 불러온다.

model(src_sample)


<tf.Tensor: shape=(128, 14, 12001), dtype=float32, numpy=
array([[[-6.95941781e-05,  1.74619128e-08, -1.75924433e-04, ...,
          1.08085420e-04,  4.28343956e-05,  1.44931866e-04],
        [-2.53788749e-04, -5.53594073e-05, -4.82216070e-04, ...,
          1.04496954e-04,  5.98956649e-05,  2.57173058e-04],
        [-2.83965375e-04, -2.02226583e-04, -6.38117082e-04, ...,
          9.31035102e-05, -1.68621639e-07,  2.45723058e-04],
        ...,
        [ 4.05087689e-04, -1.23412767e-03, -1.14462606e-03, ...,
         -1.14924667e-04, -1.41123706e-03,  1.12687564e-03],
        [ 6.03283173e-04, -1.39602239e-03, -1.38560834e-03, ...,
         -1.10887369e-04, -1.73436734e-03,  1.24715548e-03],
        [ 7.76912493e-04, -1.54053012e-03, -1.60550291e-03, ...,
         -1.30000262e-04, -2.02825037e-03,  1.34720991e-03]],

       [[-6.95941781e-05,  1.74619128e-08, -1.75924433e-04, ...,
          1.08085420e-04,  4.28343956e-05,  1.44931866e-04],
        [-1.52170396e-04,  1.01160098e-04, -3

In [18]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  1536128   
                                                                 
 lstm_2 (LSTM)               multiple                  394240    
                                                                 
 lstm_3 (LSTM)               multiple                  525312    
                                                                 
 dense_1 (Dense)             multiple                  3084257   
                                                                 
Total params: 5,539,937
Trainable params: 5,539,937
Non-trainable params: 0
_________________________________________________________________


# 모델 학습

In [19]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(dataset, epochs=1, validation_data = (enc_val, dec_val))

1220/1220 [==============================] - 1332s 1s/step - loss: 3.6818 - val_loss: 3.3281


caggle pro를 쓰지만!!!!!!!!!!!!!!!!!!!!!!!! 노트북 사양이 너무 구린 나머지..... 임베딩사이즈랑 히든사이즈를 처음에 256 1024 를 했지만 너무 오래걸려서 값을 바꾸었고 epochs도 1로 했습니다 아마
제대로 했다면 손실값이 2.2 아래로 나왔을 겁니다!!! 
 

#모델평가

In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [31]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love , i t you you you you you <end> '

In [30]:
generate_text(model, tokenizer, init_sentence="<start> babe", max_len=30)

'<start> babe , i i t you you you <end> '

In [32]:

generate_text(model, tokenizer, init_sentence="<start> you", max_len=100)

'<start> you i t t you you you you you <end> '

In [33]:

generate_text(model, tokenizer, init_sentence="<start> i", max_len=100)

'<start> i i t t you you you you you you <end> '

you 만 나오는 진기한 광경이 펼쳐 지고있군요....

#회고

초반에 뛰어쓰기 하나때문에 몇분정도 막혔지만 
성돈님과 함께 원인을 찾아 다행이 해결했지만 

결과가 좋지못해 아쉬웠다. 

노트북 바꾸고싶다 정말 

 쪼금 재미가 있었다1